<a href="https://colab.research.google.com/github/ananya-ayasi/2022_IBM_Code_Challenge_Wildfire-Detection-and-Burn-Severity-Analysis/blob/main/03_Final__WebApp_Wildfire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This notebook covers the creation of a web application to detect fire in uploaded images.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install streamlit -q

     |████████████████████████████████| 10.1 MB 5.5 MB/s 
     |████████████████████████████████| 76 kB 4.8 MB/s 
     |████████████████████████████████| 181 kB 34.0 MB/s 
     |████████████████████████████████| 164 kB 37.3 MB/s 
     |████████████████████████████████| 111 kB 47.9 MB/s 
     |████████████████████████████████| 4.3 MB 29.4 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 131 kB 43.8 MB/s 
     |████████████████████████████████| 428 kB 50.0 MB/s 
     |████████████████████████████████| 130 kB 40.0 MB/s 
     |████████████████████████████████| 793 kB 39.9 MB/s 
     |████████████████████████████████| 381 kB 43.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.29 which is incompatible.
google-colab 1

A Python script has to be written separately in order to host it as a web application.

In [25]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import streamlit as st
import PIL
import cv2
from PIL import Image, ImageOps
import numpy as np
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from keras.preprocessing import image
from keras.models import load_model
from google.colab import files
import matplotlib.pyplot as plt
from base64 import b64decode
from tensorflow import keras
from keras.models import model_from_json
from keras.preprocessing.image import img_to_array




activities = ["About","Upload Image","Live Detection","Burn Severity Analysis"]
choice = st.sidebar.selectbox("Select Activty",activities)

if choice=='About':
  st.title("Wildfire Detection and Burn Severity Analysis Using UAV and Satellite Imagery")

  video1 = open("/content/drive/MyDrive/IBM/IBM GTSP Final.mp4", "rb") 
  st.video(video1)

  st.subheader('Introduction')
  st.write("""Climate change has been a key factor in increasing the risk and extent of wildfires. 
  Timing is highly crucial in detecting wildfires and in following sufficient mitigation steps. 
  Hence, UAVs and satellites can be used in hand with artificial intelligence and image processing 
  to alert the stakeholders including the forest department, fire department and disaster management units.""")

  img = Image.open("/content/drive/MyDrive/IBM/IBM Clip Art.jpg")
  st.image(img)

  st.subheader("Wildfire Detection")
  st.write("""The user can choose the 'Upload Image' option to upload an image from their system.
    The web application will try to predict correctly if fire can be detected in the image. 
    The web application uses a CNN model that was trained on UAV imagery of wildfires for this project.""")
  
  st.subheader("Live Detection")
  st.write("""In the actual scenario, the system will be using UAVs to surveil the perimeters of the forest. 
  However, due to limitations in time, it was not possible to access an actual UAV for live detection. Hence, we have chosen a feature wherein 
  the user can use their webcam to capture images and test whether fire is present in the image.""")

  st.subheader("Burn Severity Analysis")
  st.write("""Burn Severity describes how fire intensity affects functioning of an ecosystem in the area that has been burnt. 
  Since burn severity may vary based on the specific ecosystem within which the fire has occurred, there is currently no unambiguous 
  quantifier to objectively map burn severity. Notwithstanding this, two common indices have been proposed and used to detect and assess burn severity from remote-sensing imagery.""")

elif choice=='Upload Image':
  @st.cache(allow_output_mutation=True) #to run the following function and store the result in a local cache.
  def load_model():
    model=tf.keras.models.load_model('/content/drive/MyDrive/IBM/classifier.h5')
    return model
  with st.spinner('Model is being loaded..'):
    model=load_model()

  st.title("Wildfire Detection using UAV Imagery")
  

  file = st.file_uploader("Please upload an image", 
                          type=["jpg", "png"])


  st.set_option('deprecation.showfileUploaderEncoding', False)

  def predict(img_path, model):
    size = (64,64)    
    image = ImageOps.fit(img_path, size, Image.ANTIALIAS) #to resize the image by avoiding visual defects
    image = np.asarray(image)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #convert image from on colour space to another (Several image processing libraries have different pixel orderings.)
            
    img_reshape = img[np.newaxis,...]

    result = model.predict(img_reshape)
    
    if result[0][0] == 1:
        prediction = 'notfire'
        return False
    else:
        prediction = 'fire'
        return True


  if file is None:
    st.text("Please upload an image file")
  else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = predict(image, model)
    #score = tf.nn.softmax(predictions[0])
    st.write(predictions)
    #st.write(score)
    if predictions==True:
      st.write("""
         ## Fire Detected
         """
         )
      
    else:
      st.write("""
         ## Fire Not Detected
         """
         )

elif choice=='Live Detection':
  @st.cache(allow_output_mutation=True)
  def load_model():
    model=tf.keras.models.load_model('/content/drive/MyDrive/IBM/classifier.h5')
    return model
  with st.spinner('Model is being loaded..'):
    model=load_model()
  def predict(img_path, model):
    size = (64,64)    
    image = ImageOps.fit(img_path, size, Image.ANTIALIAS)
    image = np.asarray(image)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
    img_reshape = img[np.newaxis,...]

    result = model.predict(img_reshape)
    
    if result[0][0] == 1:
        prediction = 'notfire'
        return False
    else:
        prediction = 'fire'
        return True
  

    

  st.title("Live Detection")

  

  picture = st.camera_input("Take a picture")

  if picture:
    st.image(picture)

  image = Image.open(picture)
  
  predictions = predict(image, model)
  st.write(predictions)
  if predictions==True:
    st.write("""
         ## Fire Detected
         """
         )
      
  else:
    st.write("""
         ## Fire Not Detected
         """
         )
    
elif choice=='Burn Severity Analysis':
  st.title("Burn Severity Analysis")
  img = Image.open("/content/drive/MyDrive/IBM/SR-NBR.jpeg")
  

  st.write("""Multiple space agencies across the globe including the ESA (Europe), NASA (United States), CNSA (China), JAXA (Japan) and Roscosmos (Russia) 
  have begun to launch multiple earth-observation satellites from as early as 1957.Of particular interest is a pair of sun-synchronous satellites, known collectively as Sentinel-2. 
  Sentinel 2 was launched with the aim to monitor variability in land surface conditions with a relatively short revisit period. Sentinel-2 carries a multi-spectral imaging instrument (or MSI), 
  which is able to passively record 13 spectral bands at 4096 brightness levels. It can 'see' multiple frequencies of radiation, including the visible Red, Green and Blue frequencies. 
  This enables Sentinel-2 (and other MSI orbiting platforms) to observe earth-based phenomena at a potentially more discerning level than typical RGB measurements. 
  Moreover, combinations of these bands have been used to extract and classify land-cover using the unique reflectance characteristics of water, types of earth, concrete, etc.""")

  st.subheader("Normalized Burn Ratio")
  st.write("""Normalized Burn Ratio
  The Normalized Burn Ratio (NBR) is an index that uses the differences in the way healthy green vegetation and burnt vegetation reflect light to find burnt area. 
  It is calculated using the following Sentinel-2 bands: Near Infrared/Band 8 and Shortwave Infrared/Band 12. 
  NBR returns values between -1 and 1. Healthy green vegetation will have a high NBR value while burnt vegetation will have a low value. 
  Areas of dry, brown vegetation or bare soil will also return lower NBR values than green vegetation.""")
  
  st.subheader("Delta Normalized Burn Ratio")
  st.write("""Change in Normalized Burn Ratio - also called Delta Normalized Burn Ratio (dNBR) - 
  is calculated by subtracting the post-fire NBR value from the baseline NBR value as defined in this equation:""")

  st.write("""dNBR=NBRbaseline−NBRpost fire""")

  st.write("""The dNBR value can be more useful than the NBR alone to determine what is burnt as it shows change from the baseline state. 
  A burnt area will have a positive dNBR value while an unburnt area will have a negative dNBR value or a value close to zero.
  dNBR can also be used to describe burn severity. A higher severity fire will burn more vegetation, resulting in a higher dNBR.""")

  st.image(img)


  st.write("""The burned zones underwent a significant increase in LST- Land Surface Temperature after fire. Statistically significant differences have been detected 
    between the LST within regions of burn severity categories. More substantial changes in LST are observed in zones of greater fire severity, 
    which can be explained by the lower emissivity of combustion products found in the burned area and changes in the energy balance related to vegetation removal.""")





Overwriting app.py


In [28]:
! streamlit run app.py & npx localtunnel --port 8501

2022-05-06 09:04:41.609 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 2.738s
your url is: https://good-geckos-peel-34-71-2-111.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://34.71.2.111:8501

2022-05-06 09:07:08.570149: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-06 09:08:11.412 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/PIL/Image.py", line 2847, in open
    fp.seek(0)
AttributeError: 'NoneType' object has no attribute 'seek'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/scriptrunner/script_runner.py", line 475, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 148, in <module>
    image = Image.open(p